In [ ]:
# installing dependencies
!pip install -qq pymupdf4llm peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.6 MB/s eta 0:00:00


In [ ]:
# importing dependencies
import pymupdf4llm
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import PeftModel
from typing import Optional
import os
from google.colab import userdata
import torch

# OCR processed with LLM, NER processed with BERT

In [5]:
# Initialize NER extraction pipeline
extract_ner_with_bert = pipeline(
    "ner",
    model="jplu/tf-xlm-r-ner-40-lang",
    tokenizer=(
        'jplu/tf-xlm-r-ner-40-lang',
        {"use_fast": True}),
    framework="tf"
)

# Function to process and extract NER for any text
def extract_ner(text):
    if text and isinstance(text, str) and text.strip():  # Ensure text is a string and not None or empty/whitespace
        ner_entities = extract_ner_with_bert(text)
        return ner_entities
    else:
        return []

# Get information from the PDF
info = pymupdf4llm.to_markdown(
    doc="/content/sample_data/Md_Shahnawaz_Hossain_CV.pdf.pdf",
    page_chunks=True,
    write_images=True,
    image_path="/content/sample_data/",
    image_format="jpg",
    dpi=200,
)

# Function to print structured information with real-time NER extraction
def print_info(info):
    # Print metadata details
    print("Metadata:")
    metadata = info[0]['metadata']
    for key, value in metadata.items():
        print(f"{key}: {value}")
        # Extract and print NER for metadata
        ner_entities = extract_ner(value)
        print(f"NER for metadata: {ner_entities}")

    # Print table of contents items
    print("\nTable of Contents:")
    toc_items = info[0]['toc_items']
    for item in toc_items:
        print(f"Level: {item[0]}, Title: {item[1]}, Page: {item[2]}")
        # Extract and print NER for Table of Contents title
        ner_entities = extract_ner(item[1])
        print(f"NER for Table of Contents title: {ner_entities}")

    # Print tables information
    print("\nTables:")
    tables = info[0]['tables']
    for table in tables:
        print(f"Bounding box: {table['bbox']}, Rows: {table['rows']}, Columns: {table['columns']}")
        # Extract and print NER for tables (if necessary, from bounding box or other attributes)
        ner_entities = extract_ner(str(table['bbox']))  # or another relevant part of the table
        print(f"NER for table bounding box: {ner_entities}")

    # Print images information
    print("\nImages:")
    images = info[0]['images']
    for image in images:
        print(f"Image number: {image['number']}")
        print(f"Bounding box: {image['bbox']}")
        print(f"Transform: {image['transform']}")
        print(f"Width: {image['width']}, Height: {image['height']}")
        print(f"Color space: {image['cs-name']}, Resolution: {image['xres']}x{image['yres']}")
        # Extract and print NER for image data (if relevant)
        ner_entities = extract_ner(f"Image number {image['number']} details")
        print(f"NER for image info: {ner_entities}")

    # Print extracted text
    print("\nExtracted Text:")
    text = info[0]['text']
    print(text)
    # Extract and print NER for extracted text
    ner_entities = extract_ner(text)
    print(f"NER for extracted text: {ner_entities}")

# Call function to print info
print_info(info)

Some layers from the model checkpoint at jplu/tf-xlm-r-ner-40-lang were not used when initializing TFXLMRobertaForTokenClassification: ['dropout_38']
- This IS expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaForTokenClassification were initialized from the model checkpoint at jplu/tf-xlm-r-ner-40-lang.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForTokenClassification for predictions without further training.
Device set to use 0


Processing /content/sample_data/Md_Shahnawaz_Hossain_CV.pdf.pdf...
[                                        ] (0/6)======[======                                  ] (1/6)=======[=============                           ] (2/6)=======[====================                    ] (3/6)======[==========================              ] (4/6)=======[=================================       ] (5/6)=======[========================================] (6/6)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


]
Metadata:
format: PDF 1.4
NER for metadata: [{'entity': 'ORG', 'score': 0.7308862, 'index': 1, 'word': '▁PDF', 'start': 0, 'end': 3}, {'entity': 'ORG', 'score': 0.7175045, 'index': 2, 'word': '▁1.4', 'start': 3, 'end': 7}]
title: Md_Shahnawaz_Hossain_CV.pdf
NER for metadata: [{'entity': 'PER', 'score': 0.5786691, 'index': 1, 'word': '▁Md', 'start': 0, 'end': 2}, {'entity': 'PER', 'score': 0.7356394, 'index': 3, 'word': 'Sha', 'start': 3, 'end': 6}, {'entity': 'PER', 'score': 0.74637884, 'index': 4, 'word': 'hna', 'start': 6, 'end': 9}, {'entity': 'PER', 'score': 0.79847044, 'index': 5, 'word': 'waz', 'start': 9, 'end': 12}, {'entity': 'PER', 'score': 0.735367, 'index': 7, 'word': 'Ho', 'start': 13, 'end': 15}, {'entity': 'PER', 'score': 0.7600758, 'index': 8, 'word': 'ssa', 'start': 15, 'end': 18}, {'entity': 'PER', 'score': 0.74409777, 'index': 9, 'word': 'in', 'start': 18, 'end': 20}]
author: Shahnawaz Hossain
NER for metadata: [{'entity': 'PER', 'score': 0.995793, 'index': 1, 'wor

# OCR processed with LLM, NER processed with GPT

In [ ]:
# Function to generate the prompt for NER extraction
def generate_prompt(input_text: str, instruction: Optional[str] = None) -> str:
    text = f"### Question: {input_text}\n\n### Answer: "
    if instruction:
        text = f"### Instruction: {instruction}\n\n{text}"
    return text

# Extract information from the PDF (OCR process)
info = pymupdf4llm.to_markdown(
    doc="/content/sample_data/Md_Shahnawaz_Hossain_CV.pdf.pdf",
    page_chunks=True,
    write_images=True,
    image_path="/content/sample_data/",
    image_format="jpg",
    dpi=200,
)

# Function to process OCR-extracted text and use the Gemma model for NER-like extraction
def generate_ner(text: str):
    # Construct a prompt asking the model to extract named entities from the text
    prompt = f"### Extract all named entities (people, organizations, locations, etc.) from the following text:\n\n{text}\n\n### Named Entities:"

    # Inference using the "gemma-2b" model
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        output = merged_model.generate(**inputs, generation_config=generation_config)
        response = tokenizer.decode(output[0], skip_special_tokens=True)

    return response

# Initialize model for further analysis
hugging_face_token = os.getenv('hugging_face_token')
if not hugging_face_token:
    hugging_face_token = userdata.get('hugging_face_token')
    os.environ["hugging_face_token"] = hugging_face_token

base_model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", token=hugging_face_token)
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b", token=hugging_face_token)

lora_model = PeftModel.from_pretrained(base_model, "vdpappu/lora_medicalqa")
merged_model = lora_model.merge_and_unload()

eos_token = '<eos>'
eos_token_id = tokenizer.encode(eos_token, add_special_tokens=False)[-1]

generation_config = GenerationConfig(
    eos_token_id=tokenizer.eos_token_id,
    min_length=5,
    max_length=1000,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    top_k=50,
    repetition_penalty=1.5,
    no_repeat_ngram_size=3,
    early_stopping=True
)

# Function to print structured information with real-time NER extraction using Gemma
def print_info(info):
    # Print metadata details
    print("Metadata:")
    # Extract and process Metadata
    metadata = info[0]['metadata']
    for key, value in metadata.items():
        print(f"{key}: {value}")
        # Extract and print NER for metadata
        ner_entities = generate_ner(value)
        print(f"NER for metadata: {ner_entities}")

    # Print table of contents items
    print("\nTable of Contents:")
    # Extract and process Table of Contents (TOC) items
    toc_items = info[0]['toc_items']
    for item in toc_items:
        print(f"Level: {item[0]}, Title: {item[1]}, Page: {item[2]}")
        # Extract and print NER for Table of Contents title
        ner_entities = generate_ner(item[1])
        print(f"NER for Table of Contents title: {ner_entities}")

    # Print tables information
    print("\nTables:")
    # Extract and process Tables
    tables = info[0]['tables']
    for table in tables:
        print(f"Bounding box: {table['bbox']}, Rows: {table['rows']}, Columns: {table['columns']}")
        # Extract and print NER for tables (if necessary, from bounding box or other attributes)
        ner_entities = generate_ner(str(table['bbox']))  # or another relevant part of the table
        print(f"NER for table bounding box: {ner_entities}")

    # Print images information
    print("\nImages:")
    # Extract and process Images
    images = info[0]['images']
    for image in images:
        print(f"Image number: {image['number']}")
        print(f"Bounding box: {image['bbox']}")
        print(f"Transform: {image['transform']}")
        print(f"Width: {image['width']}, Height: {image['height']}")
        print(f"Color space: {image['cs-name']}, Resolution: {image['xres']}x{image['yres']}")
        # Extract and print NER for image data (if relevant)
        ner_entities = generate_ner(f"Image number {image['number']} details")
        print(f"NER for image info: {ner_entities}")

    # Print extracted text
    print("\nExtracted Text:")
    # Extract and process Extracted Text
    text = info[0]['text']
    print(text)
    # Extract and print NER for extracted text
    ner_entities = generate_ner(text)
    print(f"NER for extracted text: {ner_entities}")

# Call the function to process and print the info with NER extraction
print_info(info)

Processing /content/sample_data/Md_Shahnawaz_Hossain_CV.pdf.pdf...
[                                        ] (0/6)======[======                                  ] (1/6)=======[=============                           ] (2/6)=======[====================                    ] (3/6)======[==========================              ] (4/6)=======[=================================       ] (5/6)=======[========================================] (6/6)]


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

adapter_config.json:   0%|          | 0.00/640 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/3.70M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Metadata:
format: PDF 1.4
NER for metadata: ### Extract all named entities (people, organizations, locations, etc.) from the following text:

PDF 1.4

### Named Entities: Person, Organization, Location, and Others

The purpose of this assignment is to give you practice extracting information about people in a natural language document such as an article or book chapter that contains some kind of reference structure for other humans who are mentioned throughout it. The goal here isn't just parsing out names but also understanding how they fit into what has been written so far; we want them treated appropriately when discussing events later on down the line! There will be three tasks associated with doing these sorts of things successfully - each requiring different types skills than those required previously-but I believe once everything comes together then everyone should feel confident tackling similar problems going forward without too much trouble getting started again if needed).

